# BOJ에서 유저가이 푼 문제 목록 정리

In [4]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import bottleneck as bn
import pickle
import time 
import random
import math
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

def add_ids(page_num : int, user_ids: list, sample_len : int = 5):
    data = requests.get(f'https://www.acmicpc.net/ranklist/{page_num}', headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    trs = soup.select('tbody > tr')

    user_len = len(trs) # 최대 100명 임
    
    if user_len < sample_len :
        sample_len = user_len

    sample_nums = random.sample(range(0,user_len),sample_len)

    for num in sample_nums:
        tr = trs[num]
        user_ids.append(tr.select_one('td:nth-child(2) > a').text)
        
        
def add_group_ids(group_num,page_num, user_ids : list):
    data = requests.get(f'https://www.acmicpc.net/school/ranklist/{group_num}/{page_num}', headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    trs = soup.select('tbody > tr')

    for tr in trs:
        user_ids.append(tr.select_one('td:nth-child(2) > a').text)

def add_to_user_problem_mat(idx, id, user_problem_mat : np.array):
    data = requests.get(f'https://www.acmicpc.net/user/{id}', headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    trs = soup.select('div.problem-list')

    for tr in trs:
        problem_nums = tr.select('a')
            
        for problem_num in problem_nums :

            problem_num = int(problem_num.text) - 1000
            #print(problem_num)
            try:
                user_problem_mat[idx,problem_num] = 1
            except:
                print("문제 번호 : " + str(problem_num))
                
def gen_user_problem_mat(id, user_problem : dict, problem_num_set : set):
    data = requests.get(f'https://www.acmicpc.net/user/{id}', headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    trs = soup.select('div.problem-list')

    time.sleep(0.1)
    
    user_problem[id] = []
    
    for tr in trs:
        problem_nums = tr.select('a')
        for problem_num in problem_nums :
            problem_num = int(problem_num.text)
            #print(problem_num)
            user_problem[id].append(problem_num)
            problem_num_set.add(problem_num)
                
def add_to_tag_pb(pb_nums : str, tag_pb : dict, tag_name_to_id : dict, num_problem: int):    
    resp = requests.get(f"https://solved.ac/api/v3/problem/lookup?problemIds={pb_nums}", headers = {'Content-Type': 'application/json'})
    
    if resp.status_code != 200: 
        print(f'{resp.status_code} : {pb_num} - error!!!')
        return
    
    jsons = resp.json()
    #print(resp)
    time.sleep(1)
    
    for json in jsons:
        tags = json['tags']
        pb_num = json['problemId']
        tag_pb[pb_num] = []
        for tag in tags:
            for displayName in tag['displayNames']:
                if displayName['language'] == 'ko' :
                    tag_name = tag['displayNames'][0]['name']
                    break
            tag_pb[pb_num].append(tag_name)
            tag_name_to_id[tag_name] = tag['bojTagId']
    

In [8]:
st_pb_num = 1000
ed_pb_num = 27917
num_problem = ed_pb_num - st_pb_num + 1 # 1000 ~ 27917

# 태그 분석

In [21]:
# 태그-문제 매트릭 생성
tag_pb = {}
tag_name_to_id = {}
num_sample_pbs = num_problem
num_slice = 100
#rand_pbs = np.random.randint(st_pb_num, ed_pb_num, num_sample_pbs)
rand_pbs = list(range(st_pb_num,ed_pb_num+1))

In [22]:
for idx in tqdm(range(0,num_sample_pbs,num_slice)) :
    rand_pbs_str = ','.join(str(e) for e in rand_pbs[idx:idx+num_slice])
    add_to_tag_pb(rand_pbs_str, tag_pb, tag_name_to_id, num_problem)    

 72%|█████████████████████████████████████████████████████████▍                      | 194/270 [03:58<01:33,  1.23s/it]


UnboundLocalError: local variable 'pb_num' referenced before assignment

In [23]:
tags = tag_name_to_id.keys()
print(len(tag_pb.keys()))
tag_pb_mat = {}
for pb in tag_pb.keys():
    tag_pb_mat[pb] = {tag:0 for tag in tags}
    for tag in tag_pb[pb]:
        tag_pb_mat[pb][tag] = 1    

18745


In [58]:
tag_problem_mat = pd.DataFrame(tag_pb_mat)

In [59]:
tag_problem_mat.to_csv("./tag_problem_mat_all.csv")

In [13]:
## 태그 분석 ##

# pd.set_option('display.max_columns', None) # 모든 열 출력
# pd.options.display.max_rows = 60 # 행 출력 초기화
# pd.options.display.max_columns = 20 # 열 출력 초기화

def get_ranking(mat, selected_tags, topk = 4):
    tag_corr_ranking = {}
    for tag in selected_tags:
        tag_corr_ranking[tag]= list(mat[tag].sort_values(ascending=False).index)[1:1+topk]
    return tag_corr_ranking

selected_tags = ['그리디 알고리즘', '다이나믹 프로그래밍', '브루트포스 알고리즘', '이분 탐색',
                '너비 우선 탐색', '깊이 우선 탐색', '데이크스트라', '플로이드–워셜', '비트마스킹', '분리 집합']

tag_problem_mat = pd.read_csv("./tag_problem_mat_all.csv", index_col = 0)

In [14]:
tag_problem_mat_trans = tag_problem_mat.T
tag_problem_mat_trans = tag_problem_mat_trans[selected_tags]
tag_problem_mat = tag_problem_mat_trans.T

In [20]:
pearson = tag_problem_mat_trans.corr()

In [21]:
pearson

,그리디 알고리즘,다이나믹 프로그래밍,브루트포스 알고리즘,이분 탐색,너비 우선 탐색,깊이 우선 탐색,데이크스트라,플로이드–워셜
그리디 알고리즘,1.000000,0.005948,-0.016486,0.018372,-0.030801,0.011689,-0.018395,-0.015990
다이나믹 프로그래밍,0.005948,1.000000,-0.047955,0.000979,-0.026054,0.017356,-0.001832,-0.004565
브루트포스 알고리즘,-0.016486,-0.047955,1.000000,-0.020930,0.003810,0.006724,-0.017410,0.008516
이분 탐색,0.018372,0.000979,-0.020930,1.000000,-0.005938,-0.013510,0.003888,-0.005533
너비 우선 탐색,-0.030801,-0.026054,0.003810,-0.005938,1.000000,0.277968,0.049118,0.023449
깊이 우선 탐색,0.011689,0.017356,0.006724,-0.013510,0.277968,1.000000,-0.007983,0.014747
데이크스트라,-0.018395,-0.001832,-0.017410,0.003888,0.049118,-0.007983,1.000000,0.075196
플로이드–워셜,-0.015990,-0.004565,0.008516,-0.005533,0.023449,0.014747,0.075196,1.000000


In [22]:
get_ranking(pearson, selected_tags)

{'그리디 알고리즘': ['이분 탐색', '깊이 우선 탐색', '다이나믹 프로그래밍', '플로이드–워셜'],
 '다이나믹 프로그래밍': ['깊이 우선 탐색', '그리디 알고리즘', '이분 탐색', '데이크스트라'],
 '브루트포스 알고리즘': ['플로이드–워셜', '깊이 우선 탐색', '너비 우선 탐색', '그리디 알고리즘'],
 '이분 탐색': ['그리디 알고리즘', '데이크스트라', '다이나믹 프로그래밍', '플로이드–워셜'],
 '너비 우선 탐색': ['깊이 우선 탐색', '데이크스트라', '플로이드–워셜', '브루트포스 알고리즘'],
 '깊이 우선 탐색': ['너비 우선 탐색', '다이나믹 프로그래밍', '플로이드–워셜', '그리디 알고리즘'],
 '데이크스트라': ['플로이드–워셜', '너비 우선 탐색', '이분 탐색', '다이나믹 프로그래밍'],
 '플로이드–워셜': ['데이크스트라', '너비 우선 탐색', '깊이 우선 탐색', '브루트포스 알고리즘']}

In [62]:
intersect = tag_problem_mat.dot(tag_problem_mat_trans)
#intersect.to_csv("./tag_tag_mat.csv")

In [63]:
intersect

,그리디 알고리즘,다이나믹 프로그래밍,브루트포스 알고리즘,이분 탐색,너비 우선 탐색,깊이 우선 탐색,데이크스트라,플로이드–워셜,비트마스킹,분리 집합
그리디 알고리즘,1257,144,66,59,9,41,9,1,22,23
다이나믹 프로그래밍,144,2312,82,76,37,68,38,10,211,16
브루트포스 알고리즘,66,82,1289,26,43,44,8,9,82,3
이분 탐색,59,76,26,663,19,9,16,2,10,26
너비 우선 탐색,9,37,43,19,606,170,40,11,25,16
깊이 우선 탐색,41,68,44,9,170,480,7,8,8,31
데이크스트라,9,38,8,16,40,7,348,15,12,5
플로이드–워셜,1,10,9,2,11,8,15,119,6,1
비트마스킹,22,211,82,10,25,8,12,6,409,2
분리 집합,23,16,3,26,16,31,5,1,2,258


In [64]:
Jaccard = {}
for in_tag in selected_tags:
    Jaccard[in_tag] = {}
    for out_tag in selected_tags:
        union = intersect[in_tag][in_tag] + intersect[out_tag][out_tag] - intersect[in_tag][out_tag]
        Jaccard[in_tag][out_tag] = intersect[in_tag][out_tag] / union

In [65]:
Jaccard = pd.DataFrame(Jaccard)

In [66]:
Jaccard

,그리디 알고리즘,다이나믹 프로그래밍,브루트포스 알고리즘,이분 탐색,너비 우선 탐색,깊이 우선 탐색,데이크스트라,플로이드–워셜,비트마스킹,분리 집합
그리디 알고리즘,1.000000,0.042044,0.026613,0.031703,0.004854,0.024175,0.005639,0.000727,0.013382,0.015416
다이나믹 프로그래밍,0.042044,1.000000,0.023302,0.026216,0.012843,0.024963,0.014493,0.004131,0.084064,0.006265
브루트포스 알고리즘,0.026613,0.023302,1.000000,0.013499,0.023218,0.025507,0.004911,0.006433,0.050743,0.001943
이분 탐색,0.031703,0.026216,0.013499,1.000000,0.015200,0.007937,0.016080,0.002564,0.009416,0.029050
너비 우선 탐색,0.004854,0.012843,0.023218,0.015200,1.000000,0.185590,0.043764,0.015406,0.025253,0.018868
깊이 우선 탐색,0.024175,0.024963,0.025507,0.007937,0.185590,1.000000,0.008526,0.013536,0.009081,0.043847
데이크스트라,0.005639,0.014493,0.004911,0.016080,0.043764,0.008526,1.000000,0.033186,0.016107,0.008319
플로이드–워셜,0.000727,0.004131,0.006433,0.002564,0.015406,0.013536,0.033186,1.000000,0.011494,0.002660
비트마스킹,0.013382,0.084064,0.050743,0.009416,0.025253,0.009081,0.016107,0.011494,1.000000,0.003008
분리 집합,0.015416,0.006265,0.001943,0.029050,0.018868,0.043847,0.008319,0.002660,0.003008,1.000000


In [104]:
new_Jaccard = {}
for in_tag in selected_tags:
    new_Jaccard[in_tag] = {}
    for out_tag in selected_tags:
        new_Jaccard[in_tag][out_tag] = intersect[in_tag][out_tag] / intersect[in_tag][in_tag]

In [105]:
new_Jaccard = pd.DataFrame(new_Jaccard)

In [106]:
new_Jaccard.T

,그리디 알고리즘,다이나믹 프로그래밍,브루트포스 알고리즘,이분 탐색,너비 우선 탐색,깊이 우선 탐색,데이크스트라,플로이드–워셜,비트마스킹,분리 집합
그리디 알고리즘,1.000000,0.114558,0.052506,0.046937,0.007160,0.032617,0.007160,0.000796,0.017502,0.018298
다이나믹 프로그래밍,0.062284,1.000000,0.035467,0.032872,0.016003,0.029412,0.016436,0.004325,0.091263,0.006920
브루트포스 알고리즘,0.051202,0.063615,1.000000,0.020171,0.033359,0.034135,0.006206,0.006982,0.063615,0.002327
이분 탐색,0.088989,0.114630,0.039216,1.000000,0.028658,0.013575,0.024133,0.003017,0.015083,0.039216
너비 우선 탐색,0.014851,0.061056,0.070957,0.031353,1.000000,0.280528,0.066007,0.018152,0.041254,0.026403
깊이 우선 탐색,0.085417,0.141667,0.091667,0.018750,0.354167,1.000000,0.014583,0.016667,0.016667,0.064583
데이크스트라,0.025862,0.109195,0.022989,0.045977,0.114943,0.020115,1.000000,0.043103,0.034483,0.014368
플로이드–워셜,0.008403,0.084034,0.075630,0.016807,0.092437,0.067227,0.126050,1.000000,0.050420,0.008403
비트마스킹,0.053790,0.515892,0.200489,0.024450,0.061125,0.019560,0.029340,0.014670,1.000000,0.004890
분리 집합,0.089147,0.062016,0.011628,0.100775,0.062016,0.120155,0.019380,0.003876,0.007752,1.000000


In [97]:
jac = np.array(new_Jaccard)

In [98]:
np.fill_diagonal(jac,0)

In [100]:
np.max(jac, axis=1)

array([0.11455847, 0.09126298, 0.06361521, 0.11463047, 0.28052805,
       0.35416667, 0.11494253, 0.12605042, 0.51589242, 0.12015504])

In [107]:
get_ranking(new_Jaccard, selected_tags)

{'그리디 알고리즘': ['다이나믹 프로그래밍', '브루트포스 알고리즘', '이분 탐색', '깊이 우선 탐색'],
 '다이나믹 프로그래밍': ['비트마스킹', '그리디 알고리즘', '브루트포스 알고리즘', '이분 탐색'],
 '브루트포스 알고리즘': ['다이나믹 프로그래밍', '비트마스킹', '그리디 알고리즘', '깊이 우선 탐색'],
 '이분 탐색': ['다이나믹 프로그래밍', '그리디 알고리즘', '브루트포스 알고리즘', '분리 집합'],
 '너비 우선 탐색': ['깊이 우선 탐색', '브루트포스 알고리즘', '데이크스트라', '다이나믹 프로그래밍'],
 '깊이 우선 탐색': ['너비 우선 탐색', '다이나믹 프로그래밍', '브루트포스 알고리즘', '그리디 알고리즘'],
 '데이크스트라': ['너비 우선 탐색', '다이나믹 프로그래밍', '이분 탐색', '플로이드–워셜'],
 '플로이드–워셜': ['데이크스트라', '너비 우선 탐색', '다이나믹 프로그래밍', '브루트포스 알고리즘'],
 '비트마스킹': ['다이나믹 프로그래밍', '브루트포스 알고리즘', '너비 우선 탐색', '그리디 알고리즘'],
 '분리 집합': ['깊이 우선 탐색', '이분 탐색', '그리디 알고리즘', '다이나믹 프로그래밍']}

# 유저-문제 분석

In [3]:
## 유저 랭킹 페이지에서 선택 ##
page_st = 31
page_ed = 250

user_ids = []

for num in tqdm(range(page_st, page_ed+1)):
    add_ids(num, user_ids,10)

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [02:36<00:00,  1.40it/s]


In [9]:
start = time.time()

user_problem_mat = np.zeros((len(user_ids),num_problem))
user_problem_mat_y = np.zeros((len(user_ids)))

for idx, id in tqdm(enumerate(user_ids), position=0):
    add_to_user_problem_mat(idx, id, user_problem_mat)

end = time.time()

print(f"{end - start:.5f} sec")

10it [00:04,  2.04it/s]

문제 번호 : 26930
문제 번호 : 26931
문제 번호 : 26932
문제 번호 : 26926


17it [00:08,  2.26it/s]

문제 번호 : 26939
문제 번호 : 26940
문제 번호 : 26941
문제 번호 : 26943


28it [00:12,  2.28it/s]

...

문제 번호 : 26918
문제 번호 : 26959


87it [00:40,  2.36it/s]

문제 번호 : 26959
문제 번호 : 26960
문제 번호 : 26961


95it [00:43,  2.17it/s]


KeyboardInterrupt: 

In [ ]:
X_train, X_test, _, _ = train_test_split(user_problem_mat, 
                                            y, 
                                            test_size=0.3, 
                                            shuffle=True, 
                                            random_state=1004)

In [ ]:
idx_to_id = {}
id_to_idx = {}
for i, id in enumerate(user_ids):
    idx_to_id[i] = id
    id_to_idx[id] = i
    
df_train = pd.DataFrame(train_user_problem_mat)
df_train.to_csv("./train_user_problem_mat.csv")
#df_rename = df.rename(index=idx_to_id) #행(row) 이름 바꾸기
#df_rename.to_csv("./test_user_problem_mat.csv")

In [111]:
## 한 그룹 내에서 선택 ##
group_num = 302 # 서울시립대학교
page_st = 1
page_ed = 4

user_ids = []

for num in tqdm(range(page_st, page_ed+1)):
    add_group_ids(group_num,num, user_ids)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.75it/s]


In [116]:
print(len(user_ids))

400


In [117]:
import math
import time

start = time.time()

user_problem = {}
problem_num_set = set([])

for id in tqdm(user_ids, position=0):
    gen_user_problem_mat(id, user_problem, problem_num_set)

end = time.time()

print(f"{end - start:.5f} sec")

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:36<00:00,  1.85it/s]

216.28883 sec


In [118]:
user_problem_mat = {}
for user, problems in user_problem.items():
    user_problem_mat[user] = {num:0 for num in problem_num_set}
    for problem in problems:
        user_problem_mat[user][problem] = 1    

In [119]:
df = pd.DataFrame(user_problem_mat).T

In [120]:
df

,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,27924,27925,27930,27931,27939,27940,27941,27942,27943,27944
iknoom1107,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
young920503,1,1,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
ljwljw8541,1,1,1,1,1,0,0,0,1,1,...,0,0,1,1,1,0,0,0,0,0
powergee,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
vjerksen,1,1,1,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kriss1607,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
mk020408,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
leejoon924,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
dlatldud11,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
df.to_csv("./uos_user_problem_mat.csv")

# EASE 사용

In [52]:
class EASE:
    """
    Embarrassingly Shallow Autoencoders model class
    """

    def __init__(self, lambda_):
        self.B = None
        self.lambda_ = lambda_

    def train(self, interaction_matrix):
        """
        train pass
        :param interaction_matrix: interaction_matrix
        """
        G = interaction_matrix.T @ interaction_matrix
        diag = list(range(G.shape[0]))
        G[diag, diag] += self.lambda_
        P = np.linalg.inv(G)

        # B = P * (X^T * X − diagMat(γ))
        self.B = P / -np.diag(P)
        min_dim = min(*self.B.shape)
        self.B[range(min_dim), range(min_dim)] = 0

    def forward(self, user_row):
        """
        forward pass
        """
        return user_row @ self.B

### 모델 생성 및 훈련

In [122]:
df_read = pd.read_csv("./uos_user_problem_mat.csv", index_col = 0)
df = df_read.to_numpy()
#print(df.shape)

In [60]:
df.shape

(532, 5828)

In [123]:
start = time.time()

ease = EASE(300)

ease.train(df)

end = time.time()

print(f"{end - start:.5f} sec")
# 훈련은 약 5분 걸림 - 유저 10000명 기준 
# 훈련은 약 10초 걸림 - 유저 500명 기준 - 시립대 학생들이 푼 문제만 

8.05006 sec


In [124]:
# 모델 저장
import pickle
 
with open('ease_model_uos.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(ease, file)

### 훈련시킨 아이디로 추천

In [68]:
idx_to_id = {}
id_to_idx = {}
for i, id in enumerate(df_read.index):
    idx_to_id[i] = id
    id_to_idx[id] = i

In [11]:
result = ease.forward(df[:, :])

result[df.nonzero()] = -np.inf

NUM_TOP_PROBLEMS = 10
top_problems_by_user = bn.argpartition(-result, NUM_TOP_PROBLEMS, axis=1)[:, :NUM_TOP_PROBLEMS] # 값이 큰 10개 문제 고름
#print(top_problems_by_user)

user_result = []
problem_result =[]

for id, top_k in enumerate(top_problems_by_user):
    user_result.extend([id] * NUM_TOP_PROBLEMS)
    problem_result.extend(top_k)

df_user_result = pd.DataFrame(user_result, columns=['user_id'])
df_problem_result = pd.DataFrame(problem_result, columns=['problem_id'])
df_result = pd.concat([df_user_result, df_problem_result], axis=1)

for key, value in idx_to_id.items():
    df_result.loc[df_result['user_id'] == key, 'user_id'] = value

df_infer = df_result.sort_values('user_id')
#print(df_infer)

[[ 1.04017955e+00  9.97626148e-01  1.01357094e+00 ...  3.44923556e-02
   3.49866983e-03 -5.23926379e-05]
 [ 1.01228131e+00  1.00876343e+00  9.77332439e-01 ...  1.38098141e-02
   8.32595045e-03  1.23995332e-02]
 [ 9.79785649e-01  9.97914205e-01  9.55551055e-01 ...  9.03417532e-01
   6.63374989e-03  3.03518542e-03]
 ...
 [ 1.01846681e+00  9.70463235e-01  4.20757764e-01 ... -6.41471502e-03
   1.93701038e-03  2.79663353e-03]
 [ 9.68699558e-01  1.03056757e+00  1.63787031e-01 ... -4.05621789e-03
   4.00295621e-03 -5.32716638e-04]
 [ 8.73316432e-01  9.29951586e-01  5.66061048e-01 ... -1.11989367e-02
   4.44420949e-04 -3.83457975e-04]]


In [ ]:
user_id = 'zzzzll'
df_infer.loc[df_infer['user_id'] == user_id,"problem_id"]

### 아무 아이디로 추천

In [125]:
# 모델 불러오기
df = pd.read_csv("./uos_user_problem_mat.csv", index_col = 0)

#ids = df_read.index
problems = set(df_read.columns)

problem_idx_to_num = {}
problem_num_to_idx = {}
for i, num in enumerate(problem_num_set):
    problem_idx_to_num[i] = num
    problem_num_to_idx[num] = i

with open('ease_model_uos.p', 'rb') as file:
    ease = pickle.load(file)

In [126]:
NUM_TOP_PROBLEMS = 10
user_id = 'sem1308'

In [127]:
#user_problem = np.zeros([1,num_problem])
#add_to_user_problem_mat(0,user_id,user_problem)

user_problem = {}
gen_user_problem_mat(user_id, user_problem, set([]))

user_problem_mat = np.zeros((1,len(problems)))

for problem in user_problem[user_id]:
    user_problem_mat[0][problem_num_to_idx[problem]] = 1    

result = ease.forward(user_problem_mat)

In [129]:
# 유저가 푼 문제와 비슷한 유형 추천 - 현재
result[user_problem_mat.nonzero()] = -np.inf
top_problems_by_user = bn.argpartition(-result, NUM_TOP_PROBLEMS, axis=1)[:, :NUM_TOP_PROBLEMS] # 값이 큰 10개 문제 고름

for i, problem in enumerate(top_problems_by_user[0]):
    top_problems_by_user[0][i] = problem_idx_to_num[problem]

print(top_problems_by_user)

[[10430 10718 10998 10818  2741 10172  1008  2178  1931 10952]]


In [ ]:
# 유저가 푼 문제와 비슷한 유형 추천 - 예전
result[user_problem.nonzero()] = -np.inf
top_problems_by_user = bn.argpartition(-result, NUM_TOP_PROBLEMS, axis=1)[:, :NUM_TOP_PROBLEMS] # 값이 큰 10개 문제 고름
top_problems_by_user += 1000
print(top_problems_by_user)

In [ ]:
# 태그로 추천

selected_tags = ['그리디 알고리즘', '다이나믹 프로그래밍', '브루트포스 알고리즘', '이분 탐색',
                '너비 우선 탐색', '깊이 우선 탐색', '데이크스트라', '플로이드–워셜', '비트마스킹', '분리 집합']

tag_problem_mat = pd.read_csv("./tag_problem_mat_all.csv", index_col = 0)
tag_problem_mat_trans = tag_problem_mat.T
tag_problem_mat_trans = tag_problem_mat_trans[selected_tags]
tag_problem_mat = tag_problem_mat_trans.T

In [82]:
tag_name = '그리디 알고리즘'
selected_probs_by_tag = []
for tag_pro_row in tag_problem_mat:
    if tag_problem_mat[tag_pro_row][tag_name]:
        selected_probs_by_tag.append(int(tag_pro_row)-1000)

In [100]:
idx_to_num = dict(zip(range(len(selected_probs_by_tag)),selected_probs_by_tag))

In [120]:
result[user_problem.nonzero()] = -np.inf
result = np.expand_dims(result[0][selected_probs_by_tag],axis=0)
top_idx_by_user = bn.argpartition(-result, NUM_TOP_PROBLEMS, axis=1)[:, :NUM_TOP_PROBLEMS][0] # 값이 큰 10개 문제 고름
top_problems_by_user = np.array([idx_to_num[idx] for idx in top_idx_by_user])
top_problems_by_user += 1000

In [121]:
top_problems_by_user

array([ 1080,  1439, 11399,  1931,  2212,  2839,  1105,  1339,  1082,
       10775])

In [113]:
top_problems_by_user += 1000

In [83]:
result[user_problem.nonzero()] = -np.inf
sorted(result)

[array([       -inf,        -inf,        -inf, ..., -0.00350231,
        -0.00335413, -0.00162263])]

In [12]:
result.shape

(1, 26918)

In [60]:
# 유저가 잘 풀지 않은 유형의 문제 추천 - 이건 비교적 부정확한 듯?
result[user_problem.nonzero()] = np.inf
top_problems_by_user_reverse = bn.argpartition(result, NUM_TOP_PROBLEMS, axis=1)[:, :NUM_TOP_PROBLEMS]
top_problems_by_user_reverse += 1000
print(top_problems_by_user_reverse)

[[ 2331 11931 10699 10819  2003 12851 10867 11051 12837  1735]]
